# <font color='yellow'>PA005: High Value Customer Identification (Insiders)

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import umap.umap_ as umap
# pip install umap-learn senão não funciona

from IPython.core.display     import HTML
from matplotlib import pyplot as plt

from pandas_profiling import ProfileReport 


from sklearn import cluster       as c
from sklearn import metrics       as m
from sklearn import ensemble      as en
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import manifold      as mn
from sklearn import mixture       as mx

from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from scipy.cluster import hierarchy as hc

from plotly import express as px

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

## Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Loading data
df_raw = pd.read_csv('Ecommerce.csv')
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


## Planejamento da Solução (IOT)

### Input 

1. Business Question
    - Find the more valiable clients to join a fidelity program
    - Selecionar os clientes mais valiosos para integrar um programa de Fidelização
2. Data Set
    - Sells of a e-commerce along one year.

## Output 

1. A indicação das pessoas que farão parte do programa de Insiders
    - Formato de lista: client id is / is insider
2. Relatório
    - Relatório com as perguntas de negócio:
    -Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Who are the clients elegible to join the Insiders program?
    -Quantos clientes farão parte do grupo?
    - How much customers will be part of the group?
    -Quais as principais características desses clientes ?
    - What are the mainly features of this customers?
    -Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Which percentage of revenue comes from the Insiders?
    -Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - How the reveneu is expected from the Insiders within the next couple of months?
    -Quais as condições para uma pessoa ser elegível ao Insiders ?
    -Quais as condições para uma pessoa ser removida do Insiders ?
    -Qual a garantia que o programa Insiders é melhor que o restante da base ?
    -Quais ações o time de marketing pode realizar para aumentar o faturamento?

## Tasks - Tarefas

1)Quem são as pessoas elegíveis para participar do programa de Insiders ?

    - O que é ser elegível? O que são os clientes de maior 'valor'? (resposta pelo time de negócio)
    - Faturamento:
        - Alto ticket médio
        - Alto LTV (Life Time Value)
        - Baixa Recência (tempo entre as compras)
        - Alto basket size
        - Baixa probabilidade de Churn (não renovação)
        - Alta Previsão de LTV
        - Alta propensão de compra
    - Custo
        - Baixa taxa de devolução
    - Experiência de compra
        - Média alta de avaliação
        
2)Quantos clientes farão parte do grupo?

    - Número total de clientes em relação ao grupo total
    
3)Quais as principais características desses clientes ?

    - Escrever características do cliente:
        -Idade
        -Localização
    - Escrever características dos hábitos de consumo
         - Atributos de clusterização
    
4)Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?

    - Comparar o faturamento dos Insiders em relação ao faturamento total
    - Faturamento do grupo (leva em conta o time que toma conta do grupo)
    
5)Qual a expectativa de faturamento desse grupo para os próximos meses ?

    - LTV do grupo de Insiders
    - Análise de Cohort
    
6)Quais as condições para uma pessoa ser elegível ao Insiders ?

    - Definir a periodicidade (quando o modelo vai ser rodado?)
    - A pessoa precisa ser desimilar ou não-parecido com uma pessoa do grupo

7)Quais as condições para uma pessoa ser removida do Insiders ?

    - Definir a periodicidade (quando o modelo vai ser rodado?)
    - A pessoa precisa ser similar ou parecido com uma pessoa do grupo
    
8)Qual a garantia que o programa Insiders é melhor que o restante da base ?

    - Teste A/B
    - Teste A/B Bayesiano
    - Teste de hipóteses

9)Quais ações o time de marketing pode realizar para aumentar o faturamento?

    - Desconto
    - preferência de compra
    - frete
    - visita a empresa

# <font color='green'> Tratamento inicial dos dados

In [4]:
df1 = df_raw.copy()

In [5]:
df1 = df_raw.drop(columns=['Unnamed: 8'], axis=1) # we don't know what is this feature/column

## Rename columns

In [6]:
df1.columns #checking the name of the columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
cols_new = ['invoice_no', 'stock_code', 'description','quantity', 'invoice_date','unit_price', 'customer_id', 'country']

In [8]:
df1.columns = cols_new #change to snacke case

## Data dimensions

In [9]:
print('The number of rows: {}'.format(df1.shape[0]))
print('The number of columns: {}'.format(df1.shape[1]))

The number of rows: 541909
The number of columns: 8


## Data Types

In [10]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

As colunas invoice e stock estão como objetos. Eu imaginei que fosse só mudar, mas o professor disse que deveria haver algum registro com letras no meio pra fazer com que fosse objeto (o que me diz que de alguma forma o pandas lê os dados e decide qual tipo vai ser). Tentando forçar a mudança pra inteiro, o local onde há um problema vai ser mostrado

In [11]:
#df1['invoice_no'] = df1['invoice_no'].astype(int)
#tá comentado pra não parar aqui quando roda tudo

'C536379' esse é o registro errado, mas ainda não é hora de tratar isso

## Check NA

In [12]:
df1.isna().sum() #soma os NAN das colunas

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

Um número muito grande de clientes NAN. Visto que estamos fazendo um projeto justamente pra agrupar clientes, é um problema bem grande

## Replace NA

First, we will separe the dataset in two, one with the customers, and another without

In [13]:
df1_missing = df1[df1['customer_id'].isna()]
df1_not_missing = df1[~df1['customer_id'].isna()]

The main idea was search for the invoices in the missing list on the another list, and identify the customers for the invoices.

In [14]:
missing_invoice = df1_missing['invoice_no'].drop_duplicates().tolist() # a lit with the invoices without customers
df1_not_missing[df1_not_missing ['invoice_no'].isin(missing_invoice)]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


But the list returned empty. We have to try another thing. Another way is give a customer id for the missing invoices, due the identification is just a code

In [15]:
df_backup = pd.DataFrame (df1_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange (19000, 19000 + len(df_backup)) 
# the max client number is 18287. So we decide start with 19000

In [16]:
#merge the two data frames
df1 = pd.merge (df1, df_backup, on = 'invoice_no', how = 'left') 
#this comand creates two  new columns in the df, witch will be dropped

In [17]:
#coalesce

df1['customer_id'] = df1['customer_id_x'].combine_first (df1['customer_id_y'] )

df1 = df1.drop (columns = ['customer_id_x', 'customer_id_y'], axis=1)

## Change dtypes

In [18]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [19]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')
#datetime é um método que gera uma data a partir de uma string
df1['customer_id'] = df1['customer_id'].astype(int64)

Tem que casar o 'format' com o formato dos dados. No df esta dia-mês-ano, mas o mês está em string, por isso ele usou o '%b'(https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [20]:
df1.dtypes #verificando se funcionou

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## Descriptive Statistics

In [21]:
num_attributes = df1.select_dtypes ( include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes ( exclude = ['int64', 'float64', 'datetime64[ns]'])

### Numerical Attributes

In [22]:
#cental tendency - mean, median
ct1 = pd.DataFrame ( num_attributes.apply (np.mean) ).T
ct2 = pd.DataFrame ( num_attributes.apply (np.median)).T
                    
# dispersion - desvio padrão, mínimo, máximo, range, skew, kurtosis
d1 = pd.DataFrame ( num_attributes.apply (np.std)).T
d2 = pd.DataFrame ( num_attributes.apply (np.min)).T
d3 = pd.DataFrame ( num_attributes.apply (np.max)).T
d4 = pd.DataFrame ( num_attributes.apply (lambda x: x.max() - x.min())).T
d5 = pd.DataFrame ( num_attributes.apply (lambda x: x.skew() )).T
d6 = pd.DataFrame ( num_attributes.apply (lambda x: x.kurtosis() ) ).T

#concatenate

df_metrics = pd.concat ([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
df_metrics.columns = ['attributes', 'min.','max','range', 'mean', 'median', 'std', 'skew', 'kurtosis']
df_metrics

,attributes,min.,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


#### <font color= 'fucsia' > Numerical Attributes - Investigating

1. Negative quantities (it can be devolutions)
2. Unit price equals zero (it can be a promotion)

#não é o momento de arrumar. Tem que ser feito tudo de uma vez, em uma seção, para deixar a estrutura lógica do storetelling

### Categorical Attibutes

***Invoice Number***

In [23]:
#quem são os invoices que contém letras?
df_letter_invoices = df1.loc[df1['invoice_no'].apply (lambda x: bool(re.search ( '[^0-9]+', x))), :] 
print('Total number of invoices with letters: {}'.format (len (df_letter_invoices)))

print('Total number of invoices with negative quantity: {}'.format( len(df_letter_invoices['quantity'] <0)))

Total number of invoices with letters: 9291
Total number of invoices with negative quantity: 9291


Estamos assumindo que todas os pedidos com letras têm quantidade negativa, por isso serão desconsiderados. Entretanto, temos quantidades negativas no data set que não possuem letras no pedido, as quantidades estão dando diferentes, mas tentei localizar e não encontrei

***Stock Code***

Não temos que fazer nenhuma conversão aqui, apenas entender a feature

In [24]:
# Tipos de Registros compostos apenas por strings que aparecem no codigo de estoque 
#(como os códigos são mistos, o Meigarom procurou códigos formados apenas por strings pra encontrar sujeiras

df1.loc[df1['stock_code'].apply (lambda x:bool (re.search('^[a-zA-Z]+$', x ) ) ), 'stock_code' ].unique()


array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

***Decription***

Não enxergamos nada pra fazer aqui. Apenas deletar a feature

***Country***

In [25]:
#Quantos clientes temos em cada país?
df1[['customer_id', 'country'
     ]].drop_duplicates().groupby('country').count().reset_index().sort_values(
         'customer_id', ascending=False).head(10)

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31
3,Belgium,25
33,Switzerland,24
27,Portugal,20
19,Italy,15
16,Hong Kong,15


# <font color='purple'>Filtragem de Variáveis

In [26]:
df2 = df1.copy() 

In [27]:
df2.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


From here, we have to take a direction. The negative values problably are from returns. We can just exclude them, exclude the in an out or we can create features for returns. From this point, we will apart the dataframe in two, one with the purchases, one just with the returns.

In [28]:
#Numerical Attributes

df2 = df2.loc[df2['unit_price'] > 0.04, :] #valor mais baixo que faz sentido dentro do df


#Categorical Attributes

In [29]:
#stock code

df2 = df2[~df2['stock_code'].isin (['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

#description

df2 = df2.drop(columns = 'description', axis=1)

#coutry

df2 = df2[~df2.isin(['European Community', 'Unspecified'])]


#são numéricos, mas vamos tirar depois dos outros filtros
#quantity
df2_returns = df2.loc[df2['quantity'] < 0, :]

df2_purchases = df2.loc[df2['quantity'] >=0, :]

#bad users

#Preciso ver como ele calculou o avg_ticket, pois esse outlier não aparece pra mim
df2 = df2[~df2['customer_id'].isin ([16446])]
#16446 comprou e devolveu tudo

# <font color='orange'>Feature Engineering

In [30]:
df3 = df2.copy()
df3_purchases = df2_purchases.copy()
df3_returns = df2_returns.copy()

## Feature Creation

In [31]:
#df_ref é um df para trabalhar
df_ref = df3_purchases.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date','unit_price','country'], axis=1).drop_duplicates(ignore_index=True)
#jogando tudo fora que eu não vou usar agora
#reseta o index sem criar uma coluna nova de index
df_ref.columns

Index(['customer_id'], dtype='object')

Pega as colunas cliente e gastos, agrupa os gastos por cliente e soma. Por padrão, a variável resultante do agrupamento (no caso a soma dos faturamentos) entra como novos índices, e nós queremos que entre como uma coluna de valores mesmo, uma feature nova. Por isso resetamos o índice de forma diferente do feito anteriormente

Up to now, we use the invoice number to solve the problem of missing customers, but from now this is irrelevant

In [32]:
df_ref.head(10).T #sobraram os clientes repetidos nos registros

,0,1,2,3,4,5,6,7,8,9
customer_id,17850,13047,12583,13748,15100,15291,14688,17809,15311,16098


Até a linha 8 era o mesmo cliente, e assim por diante. Devemos resetar os indices. As operações de concatenação, por exemplo, são feitas pelos índices (por padrão). Por isso é bom resetar. O professor já concatenou tabelas sem resetar os índices e dá um erros grotescos. O comando reset index na realidade criaria uma nova coluna de indice, que teria que ser excluída depois. É melhor usar outro comando, o ignore index.
Essa agora é a nossa tabela de referência com uma linha pra cada cliente. Todas as features serão concatenadas nela.

  Pelo que entendi, ele vai limpando e agrupando linha a linha. Primeiro tira todos os clientes repetidos, depois agrupa os valores gastos deixando uma linha por cliente depois agrupa na coluna concatenando tudo

### Gross Revenue

In [33]:
#Gross Revenue(Faturamento)
df3_purchases['gross_revenue'] = df3_purchases['quantity'] * df3_purchases['unit_price']
#Monetary (quanto a pessoa já gastou com a gente)
df_monetary = df3_purchases[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on ='customer_id', how='left')
#Concatena o df_ref e o df_monetary pela coluna customer id tendo a coluna da esquerda como referencia
df_ref.isna().sum()

customer_id      0
gross_revenue    0
dtype: int64

### Recency - Day from last purchase

Recencia - Em uma situação normal, faríamos a conta utilizando uma função para a data do dia, de forma que o modelo fosse se atualizando todos os dias. No nosso caso, temos uma base fechada em um período fechado, então é mais lógico usar a última compra do data frame como parâmetro para as outras

In [34]:
df_recency = df3_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
#retorna a data máxima por cliente
df_recency['recency_days'] = (df3_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy() #pra fazer uma cópia fisica
df_ref = pd.merge(df_ref,df_recency, on ='customer_id', how='left') 
# una as tabelas ref e recency baseado no customer_id com a tabela da esquerda como referência
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
dtype: int64

###  Quantity of Purchases

In [35]:
#number of purchases per customer
df_purchase = df3_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns = {'invoice_no': 'qtd_invoices'})
#o drop é pra tirar os invoices repetidos, o count é pra somar a quantidade de invoices de cada cliente
#drop vem como padrão 'keep first', dropa os duplicados de todas as colunas, exceto da primeira
df_ref = pd.merge (df_ref, df_purchase, on='customer_id', how='left')
df_ref.isna().sum()


customer_id      0
gross_revenue    0
recency_days     0
qtd_invoices     0
dtype: int64

###  Total quantity of Items purchased

In [36]:
#total number of items per customer
df_quantity = df3_purchases[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtd_items'})
#drop vem como padrão 'keep first', dropa os duplicados de todas as colunas, exceto da primeira
df_ref = pd.merge(df_ref, df_quantity, on='customer_id', how='left')
df_ref.isna().sum()
# Aqui cabe uma observação. Cometemos o erro crasso de dropar os duplicados na quantidade, o que não faz sentido nenhum. Além disso,
# contamos ao inves de somar. Esse erro só foi pego quando olhamos os outliers

customer_id      0
gross_revenue    0
recency_days     0
qtd_invoices     0
qtd_items        0
dtype: int64

###  Unique Basket Size - Quantity of distinct products purchased per invoice

In [37]:
df_ubasket = df3_purchases[['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id').agg ( n_purchase = ('invoice_no', 'nunique') , n_products = ('stock_code', 'nunique')).reset_index()

df_ubasket['avg_unique_basket_size'] = df_ubasket['n_products'] / df_ubasket['n_purchase']

df_ref = pd.merge (df_ref, df_ubasket[['customer_id', 'avg_unique_basket_size']], on = 'customer_id', how ='left')

df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
avg_unique_basket_size    0
dtype: int64

### Avg Ticket Value

In [38]:
#Avg Ticket
df_avg_ticket = df3_purchases[['customer_id','gross_revenue']].groupby('customer_id').mean().reset_index().rename( columns = {'gross_revenue': 'avg_ticket'})
df_ref = pd.merge(df_ref, df_avg_ticket, how ='left' , on='customer_id')
df_ref.isna().sum()

#tem um cliente que comprou e devolveu tudo, isso foi pego na analise do pandaprofiling. Tem que decidir o que vai ser feito com as devoluções

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
avg_unique_basket_size    0
avg_ticket                0
dtype: int64

###  AVG Recency Days

In [39]:
#avg recency days

### Frequency Purchase

In [40]:
#This feature does not count the total time of the dataset, just the time and number of invoices between the first and last
# purchase for each customer
# .agg function
df_aux = (df3_purchases[['customer_id', 'invoice_no',
               'invoice_date']].drop_duplicates().groupby('customer_id').agg(
                   max_=('invoice_date', 'max'), #nova coluna, sobre qual coluna, qual operação
                   min_=('invoice_date', 'min'),
                   days_=('invoice_date', lambda x:
                          ((x.max() - x.min()).days) + 1),
                   buy_=('invoice_no', 'count'))).reset_index()
# the '1' sum is because the minimum frequency is 1. If the customer just bought one time, the date will be the same and the difference
#will be 0

#frequency
df_aux['frequency'] = df_aux[['buy_','days_']].apply(lambda x: x['buy_'] / x['days_']if x['days_'] != 0 else 0,axis=1)
#merge
df_ref = pd.merge(df_ref,df_aux[['customer_id', 'frequency']],on='customer_id', how='left')
df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
avg_unique_basket_size    0
avg_ticket                0
frequency                 0
dtype: int64

In [41]:
df_ref['frequency'].max()

17.0

### Returns

In [42]:
#number of returns for each customer

df_returns = df3_returns[['customer_id', 'invoice_no']].groupby('customer_id').count().reset_index().rename (columns={'invoice_no':'qtd_returns'})
df_ref = pd.merge (df_ref, df_returns, how='left', on= 'customer_id')
#Surgiram alguns NaN. Acredito que seja por misturar o df_ref que é baseado no df3_purchases com o returns, que é baseado no 
#df3_returns.Se o cliente não fez nenhuma devolução, vai ficar com o NaN 
df_ref.loc[df_ref['qtd_returns'].isna(), 'qtd_returns'] = 0
df_ref.isna().sum()

#O Meigarom fez em relação ao número de produtos, eu fiz em relação ao número de ordens de devolução

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
avg_unique_basket_size    0
avg_ticket                0
frequency                 0
qtd_returns               0
dtype: int64

### Average Order Value

In [43]:
#Average amount of money spent in each order/invoice

df_ref['avg_invoice_value'] = df_ref['gross_revenue'] / df_ref['qtd_invoices']
df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
avg_unique_basket_size    0
avg_ticket                0
frequency                 0
qtd_returns               0
avg_invoice_value         0
dtype: int64

# <font color='red'>EDA(Exploratory Data Analysis)

## Univariate Analysis

## Bivariate Analysis

## Space study

### PCA

### UMAP

### t-SNE

### Tree-Based Embedding

# <font color='red'>Data Preparation

# <font color='red'>Feature Selection

# <font color='red'>Hyperparameter Fine-Tunning

## K-Means

## GMM

## Hierarchical Clustering

## DBSCAN

## Results

## Within-Cluster Sum of Square (WSS)

## Silhouette Score

# <font color='red'>Model Training

## K-Means

## Cluster Validation

# <font color='red'>Cluster Analysis

# <font color='red'>Deploy to production